# ML Pipeline Preparation

### 1. Import required Libariries and Load data from SQL Lite DB which was save from the ETL Pipeline


In [27]:
#Import File Reading Libareries 

import pandas as pd
import numpy as np

#Pickle to Save Trained Model

import pickle


#DB Saving and Reading from ETL Pipeline
from sqlalchemy import create_engine


#Regular Expression for Cleaning The text 
import re

#Natural Tool Kit libariries for Text Feature Engineering 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer


#ML libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score, recall_score, precision_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,cross_val_score, cross_validate
from sklearn.metrics import fbeta_score, make_scorer, SCORERS



In [28]:
# Load data from the Database which was stored from the ETL Pipeline
engine = create_engine('sqlite:///DisasterResponse.db')

df = pd.read_sql("SELECT * from Disaster_Response",engine)

X = df["message"].values

Y = (df[['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 
       'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 
       'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 
       'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']].values)

In [29]:
print(X[0],Y[0])

Weather update - a cold front from Cuba that could pass over Haiti [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [30]:
df.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,id
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,7
2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,9
4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12


In [31]:
#Check the Indexes to confirm the data read
X[1]

'Is the Hurricane over or is it not over'

In [32]:
#Check the Indexes to confirm the data read
Y[1]

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0])

### 2. Write a tokenization function to process your text data

In [33]:
def tokenize(text):
    
    # Converting all text to Lower , prerequisites for NLP Model as discussed in the NLP Course 
    text = text.lower()
    
    #Removing the Junk characters 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    #Tokenize all the text to words 
    words = word_tokenize(text)
    
    #Remove Stop words as they dont carry any waitage on the NLP Model 
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Stem the words
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    # Get Lemmas with POS Tagging 
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in stemmed]

    
    return lemmed

In [34]:

def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

### 3. Building the machine learning pipeline
This machine pipeline should take in the `message` column as Dependent Variable and output classification results on the other 36 categories(Independent Variables) in the given dataset. 

In [35]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator = RandomForestClassifier(n_jobs =-1)))
    ])

### 4. ML Pipeline Training
- Train/Test Splits
- Train the pipeline

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, Y) #from SkLearn
  

In [37]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test The model with below Parameters
1. f1 score
2. Precision And recall for each output category of the dataset


In [38]:
# Predict the model on the test data
y_pred = pipeline.predict(X_test)


In [39]:
# Model Results to be displayed
#Confugion matrix with Type 1 and 2 errors to be displayed
display_results(y_test[0], y_pred[0])

Labels: [0 1]
Confusion Matrix:
 [[35  0]
 [ 0  1]]
Accuracy: 1.0


### 6. Improving the model accuracy
The model Accuracy can be improved with various methods, one the best method is Grid Search
In this Grid search model look at permutations and combinations of all variables to get the best fit parameter
which will provide the better accuracy 

In [40]:

#clf = MultiOutputClassifier(estimator = RandomForestClassifier( random_state = 1, n_jobs = -1, oob_score =  True))

# Create the parameters list you wish to tune, using a dictionary if needed.

parameters = {}
parameters["clf__estimator__oob_score"] = [True]
parameters["clf__estimator__n_estimators"] = [10,20,50,100]
#parameters["clf__estimator__max_features"] = ["auto"]

#parameters["clf__estimator__min_samples_leaf"] = [5,10,20,30,50,100,150,200,300,400,500]


# Make an fbeta_score scoring object using make_scorer()
#scorer = make_scorer(fbeta_score, beta=.5, average = "micro")

# Perform grid search on the classifier using 'scorer' as the scoring method using GridSearchCV()
grid_obj = GridSearchCV(pipeline, parameters)

# Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit = grid_obj.fit(X_train,y_train)

# Get the estimator
best_clf = grid_fit.best_estimator_

/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-package

/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-package

/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-package

/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-package

/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-package

...

/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-package

/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-package

/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-package

/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-package

/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-package

In [41]:
# Make predictions using the unoptimized and model
predictions = y_pred
best_predictions = best_clf.predict(X_test)

In [42]:
display_results(y_test[0], best_predictions[0])

Labels: [0 1]
Confusion Matrix:
 [[35  0]
 [ 0  1]]
Accuracy: 1.0


In [43]:

# Report the before-and-afterscores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test[0], predictions[0])))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test[0], predictions[0], beta = 0.5, average = "micro")))
print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test[0], best_predictions[0])))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test[0], best_predictions[0], beta = 0.5, average = "micro")))

Unoptimized model
------
Accuracy score on testing data: 1.0000
F-score on testing data: 1.0000

Optimized Model
------
Final accuracy score on the testing data: 1.0000
Final F-score on the testing data: 1.0000


### 7. Test The model
Lets populate the below ML Parameters 
1. Accuracy
2. Precision
3. Recall

In [44]:
columns = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 
       'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 
       'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 
       'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

In [45]:
for i,col in enumerate(columns):
    print(col)
    accuracy = accuracy_score(y_test[i], best_predictions[i])
    precision = precision_score(y_test[i], best_predictions[i])
    recall = recall_score(y_test[i], best_predictions[i])
    print("\tAccuracy: %.4f\tPrecision: %.4f\t Recall: %.4f\n" % (accuracy, precision, recall))

related
	Accuracy: 1.0000	Precision: 1.0000	 Recall: 1.0000

request
	Accuracy: 0.9167	Precision: 1.0000	 Recall: 0.2500

offer
	Accuracy: 1.0000	Precision: 1.0000	 Recall: 1.0000

aid_related
	Accuracy: 0.9722	Precision: 0.0000	 Recall: 0.0000

medical_help
	Accuracy: 0.8056	Precision: 1.0000	 Recall: 0.4167

medical_products
	Accuracy: 1.0000	Precision: 0.0000	 Recall: 0.0000

search_and_rescue
	Accuracy: 0.9444	Precision: 1.0000	 Recall: 0.7143

security
	Accuracy: 1.0000	Precision: 1.0000	 Recall: 1.0000

military
	Accuracy: 0.9722	Precision: 0.0000	 Recall: 0.0000

child_alone
	Accuracy: 0.9444	Precision: 0.3333	 Recall: 1.0000

water
	Accuracy: 0.9444	Precision: 1.0000	 Recall: 0.6000

food
	Accuracy: 0.9167	Precision: 1.0000	 Recall: 0.4000

shelter
	Accuracy: 0.9167	Precision: 1.0000	 Recall: 0.5000

clothing
	Accuracy: 1.0000	Precision: 1.0000	 Recall: 1.0000

money
	Accuracy: 0.9444	Precision: 1.0000	 Recall: 0.3333

missing_people
	Accuracy: 1.0000	Precision: 0.0000	 Recall:

/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sandeepdiddi/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further by
1. We could try Different ML Algos
2. Add several other features other than TF-IDF

In [46]:
grid_fit.best_params_

{'clf__estimator__n_estimators': 100, 'clf__estimator__oob_score': True}

### 9. Export your model as a pickle file

In [47]:
filename = 'model_pipeline_udacity_project.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

In [49]:
#Check if the pickle load was sucessfull 

!ls

DisasterResponse.db                  app
ETL Pipeline Preparation.ipynb       categories.csv
LICENSE                              data
ML Pipeline Preparation.ipynb        messages.csv
README.md                            model_pipeline_udacity_project.pkl
Twitter-sentiment-self-drive-DFE.csv models


In [50]:
!pwd

/Users/sandeepdiddi/OneDrive/Udacity_Nanodegree/project2_ml_pipeline/Disaster-Response-with-Figure-Eight
